In [1]:
import pickle
import pandas as pd
import numpy as np

In [4]:
# import necessary libraries 
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import helpers

In [5]:
# import data
X_train = pd.DataFrame(pd.read_pickle('X_train.pkl'))
y_train = pd.DataFrame(pd.read_pickle('y_train.pkl'))
X_test = pd.DataFrame(pd.read_pickle('X_test.pkl'))

In [6]:
X_train.head()

,0
35,"Chief Justice Roberts, President Carter, Pres..."
13,Fellow-Citizens: We have assembled to rep...
26,"My friends, before I begin the expression of ..."
30,"Senator Hatfield, Mr. Chief Justice, Mr. Pres..."
16,My Fellow-Citizens: When we assembled her...


In [7]:
y_train.head()

,1
35,R
13,R
26,R
30,R
16,R


In [8]:
X_train.shape

(36, 1)

In [9]:
y_train[1].value_counts()

R     15
D     14
DR     4
F      2
W      1
Name: 1, dtype: int64

In [10]:
type(y_train)

pandas.core.frame.DataFrame

In [11]:
speech1 = X_train[0].iloc[35]
speech1

' Senator Dirksen, Mr. Chief Justice, Mr. Vice President, President Johnson, Vice President Humphrey, my fellow Americans--and my fellow citizens of the world community:     I ask you to share with me today the majesty of this moment. In the orderly transfer of power, we celebrate the unity that keeps us free.     Each moment in history is a fleeting time, precious and unique. But some stand out as moments of beginning, in which courses are set that shape decades or centuries.     This can be such a moment.     Forces now are converging that make possible, for the first time, the hope that many of man\'s deepest aspirations can at last be realized. The spiraling pace of change allows us to contemplate, within our own lifetime, advances that once would have taken centuries.     In throwing wide the horizons of space, we have discovered new horizons on earth.     For the first time, because the people of the world want peace, and the leaders of the world are afraid of war, the times are 

In [12]:
df = X_train.join(y_train)
df.head()

,0,1
35,"Chief Justice Roberts, President Carter, Pres...",R
13,Fellow-Citizens: We have assembled to rep...,R
26,"My friends, before I begin the expression of ...",R
30,"Senator Hatfield, Mr. Chief Justice, Mr. Pres...",R
16,My Fellow-Citizens: When we assembled her...,R


In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 35 to 28
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       36 non-null     object
dtypes: object(1)
memory usage: 1.8+ KB


In [15]:
X_train.columns

Int64Index([0], dtype='int64')

In [16]:
X_train.head()

,0
35,"Chief Justice Roberts, President Carter, Pres..."
13,Fellow-Citizens: We have assembled to rep...
26,"My friends, before I begin the expression of ..."
30,"Senator Hatfield, Mr. Chief Justice, Mr. Pres..."
16,My Fellow-Citizens: When we assembled her...


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

# Break into words/ remove stopwords
tfidf = TfidfVectorizer(token_pattern=r"([a-zA-Z]+(?:'[a-z]+)?)", stop_words=stopwords.words('english'))

X_train_vec = tfidf.fit_transform(X_train[0]) # y=y_train ?

X_test_vec = tfidf.transform(X_test[0])

X_train_vec = pd.DataFrame(X_train_vec.toarray(), columns=tfidf.get_feature_names())
X_train_vec.head()

,abandon,abandoned,abandonment,abate,abdicated,abhorring,abide,abiding,abilities,ability,...,yorktown,young,younger,youngest,youth,youthful,zeal,zealous,zealously,zone
0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.026797,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.030272,0.0,0.0
2,0.0,0.000000,0.0,0.000000,0.0,0.033425,0.0,0.019694,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.042176,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.026884,0.0,0.037361,0.0,0.000000,0.0,0.022013,0.0,0.019511,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [18]:
X_train_vec.sum(axis=1)

0     17.900523
1     22.796441
2     24.003194
3     23.419937
4     23.054758
5     21.505610
6     24.473929
7     18.619610
8     31.280287
9     15.761642
10    25.479738
11    25.304741
12     6.996305
13    25.178140
14    17.928739
15    20.923678
16    20.789033
17    25.374656
18    23.937296
19    23.166283
20    12.369011
21    22.239285
22    20.120361
23    19.212513
24    18.473365
25    21.341209
26    17.658210
27    26.008446
28    21.933566
29    27.104264
30    20.342647
31    17.628385
32    31.233517
33    29.540063
34    25.018982
35    20.954626
dtype: float64

In [85]:
nb_classifer = MultinomialNB()
rf_classifier = RandomForestClassifier(n_estimators=10, random_state=666)

In [72]:
# create a function that takes in a classifier, trains it on our tf-idf vectors,
# and generates train and test predictiions
def classify_text(classifier, X_train, X_test, y_train):
    
    # a) fit the classifier with our training data
    classifier.fit(X_train, y_train)
    
    # b) predict the labels of our train data and store them in train_preds
    train_preds = classifier.predict(X_train)
    
    # c) predict the labels of our test data and store them in test_preds
    test_preds = classifier.predict(X_test)
    
    # d) return train_preds and test_preds
    return train_preds, test_preds

In [73]:
nb_train_preds, nb_test_preds = classify_text(nb_classifer, X_train_vec, X_test_vec, y_train)

C:\Users\samjd\anaconda3\envs\oy-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [74]:
nb_train_preds

array(['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'D', 'D', 'R',
       'R', 'D', 'D', 'D', 'D', 'R', 'R', 'D', 'R', 'R', 'R', 'D', 'D',
       'D', 'D', 'D', 'R', 'D', 'D', 'R', 'D', 'R', 'R'], dtype='<U2')

In [75]:
nb_test_preds

array(['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R'], dtype='<U2')

In [86]:
rf_train_preds, rf_test_preds = classify_text(rf_classifier, X_train_vec, X_test_vec, y_train)

<ipython-input-72-9a22ca2a737b>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


In [77]:
rf_train_preds

array(['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'W', 'R', 'D', 'R', 'F',
       'DR', 'D', 'D', 'D', 'DR', 'D', 'F', 'D', 'DR', 'R', 'D', 'D', 'D',
       'D', 'D', 'D', 'R', 'D', 'D', 'R', 'D', 'R', 'R'], dtype=object)

In [88]:
with open('oa_sjd_predictions.pkl', 'wb') as handle:
    pickle.dump(rf_test_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('oa_sjd_predictions.pkl', 'rb') as handle:
    b = pickle.load(handle)


In [83]:
from sklearn.metrics import confusion_matrix, classification_report

In [87]:
confusion_matrix(y_train, rf_train_preds)

array([[14,  0,  0,  0,  0],
       [ 0,  4,  0,  0,  0],
       [ 0,  0,  2,  0,  0],
       [ 0,  0,  0, 15,  0],
       [ 0,  0,  0,  0,  1]], dtype=int64)